# Information Retrieval Coursework (7071CEM)

Task:
Develop a vertical search engine similar to Google Scholar that only retrieves papers/books published by a member of Coventry University. As such, at least one of the co-authors must be a Coventry University Staff member. 

To this end, the profiles of academic staff at CU available on the CU web site are crawled, and their papers within their profiles are indexed.

## Package Installs

Some packages required for this task are by default not included in the jupyter notebooks package list. A pip-install of these packages is required. 

In [ ]:
# Delete the "#" before each subsequent line to install the package

#!pip install scrapy
#!pip install requests
#!pip install BeautifulSoup4
#!pip install nltk
#!pip install gensim
#!pip install xgboost
#!pip install pandastable

## Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

# 1. Crawler Component

In [ ]:
# Define Seed Page
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"

# Define profile URL format. This was obtained by manually examining the profile pages
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

The seed page, i.e coventry researchers profiles landing page, contains profiles of all academic researchers of the University. As there are up to 2206 results, a limited number are displayed at once (50).

Even if new profiles are added and a new page is required, the crawler will need to crawl through all pages to access every profile. To do this, a function is defined to retrieve the current total number of result pages. 

In [ ]:
def get_maximum_page():
    
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
    
mx = get_maximum_page()    

Instead of crawling all researchers, this web crawler is designed to specifically find researchers who:
    
1. Have research publications
2. Are part of the "**School of Computing, Electronics and Maths**" 

In [ ]:
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        # Check department
        if span.text == str('School of Computing, Electronics and Maths'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
    database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
    database.to_csv('database.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):
    
    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")
    
    
    for paper in papers:
        title = paper.find('h3', class_='title').find('span')
        author = paper.find('a', class_='link person').find('span')
        date = paper.find('span', class_="date")
        link = paper.find('h3', class_='title').find('a', href=True)['href']
        
        opening = pd.read_csv('database.csv', index_col="Unnamed: 0")
        opening = opening.append({'Title': title.text, 
                                  'Author': author.text, 
                                  'Published': date.text,
                                  'Link': link}, ignore_index=True)
        opening.to_csv('database.csv')
                

In [ ]:
## Scrape function
def scrape(mx):
    df = update_csv('database.csv')
    i=0
    while True:
    
        if i > 17:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [ ]:
create_csv()

%time scrape(mx) 

In [ ]:
sample_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'})
sample_db
print(f'{sample_db.shape[0]} records were scraped')

# 2. Indexing Component

In [ ]:
scraped_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'}).reset_index(drop=True)
scraped_db.head()

In [ ]:
sample_db.head(7)
#ids = scraped_db["Title"]
#scraped_db[ids.isin(ids[ids.duplicated()])]

In [ ]:
single_row = scraped_db.loc[1,:].copy()
single_row

## 2.1 Preprocess Text

In [ ]:
# Remove stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()




def tp1(txt):
    txt = txt.lower()   # Make lowercase
    txt = txt.translate(str.maketrans('',
                                      '',
                                      string.punctuation))   # Remove punctuation marks
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


In [ ]:
# Sample title
single_row['Title']

In [ ]:
# Demonstration of lowercase and punctuation removal
tp1(single_row['Title'])

In [ ]:
# Demonstration of lematization

lematize(tp1(single_row['Title']))
#lematize(single_row['Title'])

#### Unprocessed

In [ ]:
sample_db['Title'].iloc[5]

#### Processed

In [ ]:
scraped_db['Title'].iloc[5]

### 2.1.1 Preprocess entire dataframe

In [ ]:
processed_db = scraped_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

## 2.2 Index Construction

In [ ]:
single = processed_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
SN = single.SN
word = words[0]
example = {word: [SN]}

print('=====================================================================')
print('Sample index')
print(example)

In [ ]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

indx = apply_index(inputs=single, index= {})
#print(indx)

In [ ]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})

In [ ]:
with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [ ]:
len(data)

In [ ]:
data

## 3.  Query Processor

In [ ]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    #print(f'User Search Query: {sample}')
    print(f'Processed Search Query: {processed_query}')
    return processed_query
    
#demonstrate_query_processing()

### 3.1.  Split Query into individual terms

In [ ]:
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

### 3.2.  Boolean Functionalities

In [ ]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

### 3.3. Search Engine Function

In [ ]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [ ]:
def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

In [ ]:
def final_engine(results):
    if type(results) != 'list':
        return results
        #print(results)
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]
            #print(printout['Title'])
            #print(printout['Author'])
            #print(printout['Published'])
            #print(printout['Link'])
            #print('')

In [ ]:
scraped_db['Title'].iloc[24]

In [ ]:
final_engine(test_search_engine())

## 4. Schedule Crawler for every week

To demonstrate a weekly scheduled crawling, the following parameters are defined:

* `interval` : Represents number of days in reality. In this code, it represents only seconds for demonstration


In [ ]:
days = 0
interval = 7
while days <= 1:
    scrape(mx)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

# 5. GUI

In [ ]:
from tkinter import *
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')


image1 = Image.open('coventry-university-logo.png')
resized_image1 = image1.resize((500,300), Image.ANTIALIAS)

def new_gui(pic):
    window = Tk()
    window.title("Coventry University Oracle")
    window.geometry('1100x600')
    
    lbl = Label(window, text="CEM Search Engine", font=("Arial Bold", 30), padx=5, pady=5)
    lbl.grid(column=1, row=0)
    
    lbl2 = Label(window, text="Enter your search query here ===>", font=("Arial", 15), padx=5, pady=5)
    lbl2.grid(column=0, row=1)
    
    
    img = ImageTk.PhotoImage(pic)
    
    lbl3 = Label(image=img)
    lbl3.image = img
    lbl3.grid(column=1, row=3, padx=5, pady=5)
    
    
    
    query = Entry(window,width=40)
    query.grid(column=1, row=1,  padx=5, pady=5)
    
    results = Canvas(window, height=30, width=250)
    results.grid(column=1, row=2, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search", command=clicked)
    btn.grid(column=2, row=1)
    
    

    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [ ]:
new_gui(image1)